In [18]:
import pandas as pd
import numpy as np
import os
import random

char_df = pd.read_csv("character_stats.csv")
weap_df = pd.read_csv("weapon_stats.csv")
# char_df.head()
# weap_df.head()



FileNotFoundError: [Errno 2] No such file or directory: 'character_stats.csv'

In [6]:
def user_input():
    #Get input from users on what models they are using, then format it so that it can be compared to the table (e.g. replace space with _ )

    attacking_models = 'scout, special weapons gunner, scout gunner' # input("What models are you using?: ")
    # scout, tactical marine, scout gunner
    attacking_models = attacking_models.split(", ")
    attacking_models = [model.replace(" ", "_") for model in attacking_models]
    # print(attacking_models)

    #Get input from user on what models are being attacking, then format it so that it can be compared to the table (e.g. replace space with _ )
    defending_models = 'kabalite_warrior, kabalite_gunner, sybarite' #input("What models are you attacking?: ")
    # kabalite_warrior, kabalite_gunner, sybarite

    defending_models = defending_models.split(", ")
    defending_models = [model.replace(" ", "_") for model in defending_models]
    # print(defending_models)

    char_index = char_df.index

    attacking_model_indexes = []
    for model in attacking_models:
        #attacking_model_indexes.append(int("".join(map(str, [number for number in str(char_index[char_df['name'] == model].tolist()) if number.isdigit()]))))
        attacking_model_indexes.append((char_index[char_df['name'] == model][0].item()))

    # print("Attackers:", attacking_model_indexes)

    defending_model_indexes = []
    for model in defending_models:
        #defending_model_indexes.append(int("".join(map(str, [number for number in str(char_index[char_df['name'] == model].tolist()) if number.isdigit()]))))
        defending_model_indexes.append((char_index[char_df['name'] == model][0].item()))

    info = {'attacking_model_indexes' : attacking_model_indexes, 'defending_model_indexes' : defending_model_indexes}
    #info = {'attacking_model_indexes' : attacking_model_indexes, 'defending_model_indexes' : defending_model_indexes, 'weapon_indexes' : weapon_indexes, 'loadout_unique_weapons' : loadout_unique_weapons}
    return info

In [7]:
def roll_die():
        result = random.randrange(1,7)  
        # print('Rolled',result)   
        return result 
    
roll_die()

4

In [8]:

def hit_roll(attacking_model_index):
    hit = False
    # roll = roll_die() #roll die 

    # print(char_df.iloc[0]["name"])
    # print(char_df.iloc[0]["ballistic_skill"])

    ballistic_skill = int(char_df.iloc[0]["ballistic_skill"][:-1])
    roll_required =  ballistic_skill # roll required to hit is the ballistic skill but get rid of the '+' which is the last character
    if roll_die() >= roll_required: # check that the roll is above ballistic skill so that the attack has actually hit
        hit = True

    return hit

In [9]:
def wound_roll(weapon_index, attacking_model_index, defending_model_index):
    # roll = roll_die() #roll die 
    wounded = False
    weap_strength = weap_df.iloc[weapon_index]["strength"] # [number] is dependent on what weapon is being tested.
    defender_toughness = int(char_df.iloc[defending_model_index]["toughness"])
    
    roll_required = None

    if weap_strength.isdigit():
        weap_strength = int(weap_strength)
        if weap_strength / defender_toughness >= 2:
            roll_required = 2

        if weap_strength > defender_toughness:
            roll_required = 3

        if weap_strength == defender_toughness:
            roll_required = 4
        
        if weap_strength < defender_toughness:
            roll_required = 5

        if weap_strength / defender_toughness <= 0.5:
            roll_required = 6

    else:
        if weap_strength == 'user':
            weap_strength = char_df.iloc[attacking_model_index]["strength"] 
            roll_required = int(weap_strength)
            
            
        # print("strength:", weap_strength, "\ntoughness:",defender_toughness)
        if weap_strength == "4+":
            roll_required = 4

    if roll_required is None:
        print("attacking model:", char_df.iloc[attacking_model_index]["name"])
        raise Exception("ROLL REQUIRED IS NONE !!!!!!!!!!!", weap_strength)
   

    if roll_die() >= roll_required:
        wounded = True

    return wounded
#find amt of wounds

# wound_roll(wounded)

In [10]:
def save_roll(weapon_index, defending_model_index):
    saved = False

    armour_penetration = weap_df.iloc[weapon_index]["armour_penetration"]
     
    save = int(char_df.iloc[defending_model_index]["save"][:-1]) 
    modified_roll = roll_die() + int(armour_penetration)

    # print("Modified Save:", modified_save)

    if modified_roll >= save: # get marco to check !!
        saved = True
    return saved

# save_roll(weapon_index, defending_model_index, saved)

In [11]:
def inflict_dmg(weapon_index, defending_model_index):
    damage = weap_df.iloc[weapon_index]['damage']

    wounds = char_df.iloc[defending_model_index]['wounds']

    # in other function we should edit the health

    # print("Damage:", damage)
    if damage.isdigit():
        damage = int(damage)  
    else: # if it isnt an int, situations such as roll d3 or d6 in the damage category, it isnt an integer
        #print("Damage is "+damage+" _________________________________________________________________________________________")
        die = int(damage[1])
        #print("Die is {0:1d} _________________________________________________________________________________________".format(die))
        if die == 3: # 1-2=1 , 3-4=2 , 5-6=3 for d3 dice
            damage = int(np.ceil(roll_die()/2)) # round up from half of the dice roll to btain the d3 dice roll tings
        elif die == 6:            
            damage = roll_die()
    # else:
    #     raise Exception("A Non'-integer has been introduced to inflict_damage as the damage value")
    return damage, wounds
# inflict_dmg()

In [12]:
def injury_roll(wounds_remaining, out_of_action, flesh_wounded, flesh_wounds):
    if wounds_remaining <= 0:
        result = roll_die()
        modified_result = result + flesh_wounds
        # print("Modified Result", modified_result)

        if modified_result > 3 or flesh_wounds >= 3:
            out_of_action = True

        elif modified_result <=3:
            flesh_wounded = True
            # flesh_wounds += 1

    return out_of_action, flesh_wounded#, flesh_wounds
# injury_roll(wounds_remaining, flesh_wounds, out_of_action)

In [13]:
def resolve_attacks(weapon_index, attacking_model_index, defending_model_index, out_of_action, flesh_wounded, flesh_wounds): #, out_of_action, flesh_wounds
    
    # print("ATTACKING MODEL:", char_df.iloc[attacking_model_index]['name'])
    # print("DEFENDING MODEL:", char_df.iloc[defending_model_index]['name'])
    # print("CURRENT WEAPON:", weap_df.iloc[weapon_index]['weapon'],"\n")
    if out_of_action == False:
        if hit_roll(attacking_model_index) == True: # check if the hit roll is hit
            # print("HIT \n")

            if wound_roll(weapon_index, attacking_model_index, defending_model_index) == True:
                # print("WOUNDED \n")

                if save_roll(weapon_index, defending_model_index) == False:
                    # print("NOT SAVED \n")

                    damage = inflict_dmg(weapon_index, defending_model_index)[0]
                    wounds = inflict_dmg(weapon_index, defending_model_index)[1]

                    
                    
                    wounds_remaining = int(wounds) - int(damage)
                    
                    injury_roll_output = injury_roll(wounds_remaining, out_of_action, flesh_wounded, flesh_wounds) #, flesh_wounds, 
                    out_of_action = injury_roll_output[0]
                    flesh_wounded = injury_roll_output[1]
                    # flesh_wounds = injury_roll_output[2]
                    # print("Fleshwounds:",flesh_wounds)

                    if flesh_wounded == True and wounds_remaining > 1:
                        wounds_remaining = 1

                    # if flesh_wounded == True:
                        # add_flesh_wounds += 1
                        # print("FLESH WOUNDED")

                    if out_of_action == True or flesh_wounds >= 3:
                        # print("OUT OF ACTION")

                        out_of_action = True

    return {'weapon_index' : weapon_index, 'attacking_model_index' : attacking_model_index, 'defending_model_index' : defending_model_index, 'out_of_action' : out_of_action, 'flesh_wounds' : 1, 'flesh_wounded' : flesh_wounded}
# resolve_attacks(weapon_index, attacking_model_index, defending_model_index, hit, wounded, saved)


In [14]:
def simulate_weapon(weapon_index, attacking_model_index, defending_model_index):
    
    turns = 0
    flesh_wounds = 0
    out_of_action = False
    flesh_wounded = False
    attacks = 2
    
    while out_of_action == False:

        for attack_iteration in range(attacks):
            
            simulation_data = resolve_attacks(weapon_index, attacking_model_index, defending_model_index, out_of_action, flesh_wounded, flesh_wounds) #, flesh_wounds,
            out_of_action = simulation_data['out_of_action']
            

            # print(simulation_data)
            # flesh_wounds+= 1
            if simulation_data['flesh_wounded'] == True:
                flesh_wounds += 1
                # print("FLESH WOUNDSSS", flesh_wounds)

            
            # print("TURNS:", turns)
            # print("------------------------------------------------------------------------------------------------------\n")
        turns+= 1
    
    # print("\n\n\nbreak break break !!!!!")
    lcl_weapon_index = simulation_data['weapon_index']
    lcl_attacking_model_index = simulation_data['attacking_model_index']
    lcl_defending_model_index = simulation_data['defending_model_index']
    results = np.array([[turns, lcl_weapon_index, lcl_attacking_model_index, lcl_defending_model_index]])
    

    # return turns, lcl_weapon_index, lcl_attacking_model_index, lcl_defending_model_index
    return results
    # turns+= 1
# simulate_weapon(weapon_index, attacking_model_index, defending_model_index, hit, wounded, saved)

In [15]:

def optimise(amt_simulations, attacking_model_index, defending_model_index, loadout_weapon_index):

    loadout_results = np.empty((0, 4), dtype=np.float16)

        # print(weapon_index)
        # print("CURRENT WEAPON:", weap_df.iloc[weapon_index]['weapon'],"\n")
    simulation_arr = np.empty((0, 4), dtype=np.int8) #initialise empty array with 4 rows

    for simulation in range(amt_simulations):
        
        results = simulate_weapon(loadout_weapon_index, attacking_model_index, defending_model_index) #this includes [[turns, lcl_weapon_index, lcl_attacking_model_index, lcl_defending_model_index]
        simulation_arr = np.append(simulation_arr, results, axis=0) # schzoom those returned values in
        
        # return turns

    # print(simulation_arr)
    # print(simulation_arr.shape)

    total = simulation_arr[: , 0].sum()

    count = simulation_arr.shape[0]
    # print("TOTAL:",total)
    # print("COUNT:",count)

    mean = total/count
    # print("MEAN:",mean)

    current_weapon_index = simulation_arr[0][1]
    current_attacking_model_index = simulation_arr[0][2]
    current_defending_model_index = simulation_arr[0][3]

    current_weapon_result = [[mean, current_weapon_index, current_attacking_model_index, current_defending_model_index]]
    loadout_results = np.append(loadout_results, current_weapon_result, axis=0)
    # print("\n -----------------------------------------------------------\n")
    
    # print(loadout_results)

    # Sort 2D numpy array by 2nd Column

    # ordered_loadout_results = loadout_results[loadout_results[:,0].argsort()] ###########################

    # print(ordered_loadout_results)
        #turns, lcl_weapon_index, lcl_attacking_model_index, lcl_defending_model_index is the order of the array
    return loadout_results

# optimise(amt_simulations, attacking_model_indexes, defending_model_indexes)


In [16]:
def generate_weapon_indexes(attacking_model_index):
    weapon_loadouts = []
    # weapon_loadouts_uc = str(np.array(char_df)[0][-1]).split("\n")
    weapon_loadouts_uc = str(np.array(char_df)[attacking_model_index][-1]).split("\n")

    for loadout in weapon_loadouts_uc:
        # print(loadout.split(", "))
        weapon_loadouts.append(loadout.split(", "))


    print('attacking_weapons:',weapon_loadouts)
    # print('loadout_unique_weapons',loadout_unique_weapons)

    all_weapon_indexes = []
    

    weap_index = weap_df.index

    for loadout in weapon_loadouts: # get a list of the weapon indexes
        loadout_indexes = []
        for weapon in loadout:
            loadout_indexes.append(int("".join(map(str, [number for number in str(weap_index[weap_df['weapon'] == weapon].tolist()) if number.isdigit()]))))
        all_weapon_indexes.append(loadout_indexes)

    return all_weapon_indexes

In [19]:
def main():
    user_info = user_input()     # info = {attacking_model_indexes : 'attacking_model_indexes', defending_model_indexes : 'defending_model_indexes', weapon_indexes : 'weapon_indexes', loadout_unique_weapons : 'loadout_unique_weapons'}
    attacking_model_indexes = user_info['attacking_model_indexes']
    defending_model_indexes = user_info['defending_model_indexes']
    # weapon_indexes = user_info['weapon_indexes']
    # loadout_unique_weapons = user_info['loadout_unique_weapons']
    

    amt_simulations = 100
    
    print("\n")

    for attacking_model_index in attacking_model_indexes:
        all_weapon_indexes = generate_weapon_indexes(attacking_model_index)
        print("all_weapon_indexes:",all_weapon_indexes)
        for defending_model_index in defending_model_indexes:
            # print("\n")
            print("ATTACKING MODEL:", char_df.iloc[attacking_model_index]['name'])
            print("DEFENDING MODEL:", char_df.iloc[defending_model_index]['name'])
            # optimal_loadouts = np.empty((0, 4), dtype=np.int8)
            all_results = np.empty((0, 4))
            for loadout_weapon_indexes in all_weapon_indexes:
                loadout_results = np.empty((0, 4))
                
                for weapon_index in loadout_weapon_indexes:
                    # print("weapon index:", weap_df.iloc[weapon_index]["weapon"])   
                    results = optimise(amt_simulations, attacking_model_index, defending_model_index, weapon_index)
                    loadout_results = np.append(loadout_results, results, axis = 0)
                    # np.append(optimal_loadouts, results, 0)

                index_minimum = np.where(loadout_results[:,0] == np.amin(loadout_results[:,0]))
                # print("index_minimum:",index_minimum)

                minimum_result = loadout_results[index_minimum[0][0]]
                # print("minimum_result:",minimum_result)
                all_results = np.append(all_results, [minimum_result], axis = 0)
                all_ordered_results = all_results[all_results[:,0].argsort()]
            
            # i = np.arange(len(all_ordered_results))
            # print([i])
            # index_column = np.append(index_column, )
            # all_ordered_results = np.append(all_ordered_results, i)
            print("all ordered results:\n",all_ordered_results)
            print("\n")
    print("FINISHED")
# main()

In [20]:
if __name__ == "__main__":
    main()

NameError: name 'char_df' is not defined